In [271]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from io import StringIO, BytesIO
from azure.storage.blob import BlobServiceClient
import pandas as pd
from datetime import datetime
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
import numpy as np

## Azure authentication
Don't forget to `az login` in cmd!

In [272]:
keyVaultName = "cvprojectkeyvault"
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [273]:
FACE_KEY = "c819ffcc7744491f900087e3768046fe"
FACE_ENDPOINT = "https://cvprojectface2.cognitiveservices.azure.com/"

In [274]:
# Create a client
face_client = FaceClient(FACE_ENDPOINT, CognitiveServicesCredentials(FACE_KEY))

In [275]:
face_client.api_version

'1.0'

## Check Flight Manifest table before validation

In [333]:
account_url = "https://computervisionproject.blob.core.windows.net"
blob_service_client = BlobServiceClient(account_url, credential=credential)
container = 'kiosk'
blob_path = 'manifest.csv'
blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)

In [334]:
downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
blobstring = downloader.readall()

In [335]:
df_manifest = pd.read_csv(StringIO(blobstring))
df_manifest

,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,False,False,False,False,False
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,False,False,False,False,False
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,False,False,False,False,False
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,False,False,False,False,False


## Functions to perform full validation for one passenger

In [279]:
def get_id_card_infos():
    container = 'kiosk'
    blob_path = 'extracted_id_details/id_details.parquet'
    blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    stream = BytesIO()
    downloader.readinto(stream)
    df_id = pd.read_parquet(stream, engine='pyarrow')
    # Convert all names to lowercase
    df_id['first_name'] = df_id['first_name'].str.lower()
    df_id['last_name'] = df_id['last_name'].str.lower()
    # Workaround for Avkash's ID (ignore second part of first name)
    df_id['first_name'] = df_id['first_name'].apply(lambda x: x.split(' ')[0])
    return df_id

In [280]:
def get_boarding_pass_infos():
    container = 'kiosk'
    blob_path = 'extracted_boarding_pass_details/boarding_pass_details.parquet'
    blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    stream = BytesIO()
    downloader.readinto(stream)
    df_bp = pd.read_parquet(stream, engine='pyarrow')
    return df_bp

In [281]:
def get_lighter_detection_infos():
    container = 'kiosk'
    blob_path = 'lighter_detection/predictions.parquet'
    blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    stream = BytesIO()
    downloader.readinto(stream)
    df_ld = pd.read_parquet(stream, engine='pyarrow')
    return df_ld

In [352]:
def validate_name(manifest, first_name, last_name, debug=False):
    # Check manifest name
    df_passenger = df_manifest[(df_manifest['FirstName'].str.lower()==first_name) & (df_manifest['LastName'].str.lower()==last_name)]
    if df_passenger.empty:
        print("Passenger " + first_name + " " + last_name + " not found in Flight Manifest.")
        return False
    # Check ID card name
    df_id = get_id_card_infos()
    df_pass_id = df_id[(df_id['first_name']==first_name.lower()) & (df_id['last_name']==last_name.lower())]
    if df_pass_id.empty:
        print("Dear Sir/Madam,\nSome of the information on your ID card does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    # Check boarding pass name
    df_bp = get_boarding_pass_infos()
    df_pass_bp = df_bp[df_bp['Passenger Name'].str.lower()==first_name + " " + last_name]
    if df_pass_bp.empty:
        print("Dear Sir/Madam,\nSome of the information in your boarding pass does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    if debug:
        print("3-Way Person Name Validation successful for passenger " + first_name + " " + last_name)
    return True

In [353]:
def validate_dob(manifest, first_name, last_name, debug=False):
    # Check date of birth on flight manifest
    dob_manifest = datetime.strptime(list(df_manifest.loc[(df_manifest['FirstName'].str.lower()==first_name) & (df_manifest['LastName'].str.lower()==last_name), 'DateOfBirth'])[0], '%Y-%m-%d')
    dob_manifest = dob_manifest.date()
    # Check date of birth on ID card
    df_id = get_id_card_infos()
    dob_id = list(df_id.loc[(df_id['first_name']==first_name) & (df_id['last_name']==last_name), 'dob'])[0]
    if dob_manifest!=dob_id:
        print("Dear Sir/Madam,\nSome of the information on your ID card does not match the flight manifest data, so you cannot board the plane.\nPlease see a customer service representative.")
        return False
    else:
        if debug:
            print("Date of Birth Validation successful for passenger " + first_name + " " + last_name)
        return True

In [379]:
def validate_boarding_pass(manifest, first_name, last_name, debug=False):
    df_pass_m = df_manifest[(df_manifest['FirstName'].str.lower()==first_name) & (df_manifest['LastName'].str.lower()==last_name)].reset_index()
    df_bp = get_boarding_pass_infos()
    df_pass_bp = df_bp[df_bp['Passenger Name'].str.lower()==first_name + " " + last_name].reset_index()
    bp_valid = True
    if(str(df_pass_m.loc[0,'FlightNo'])!=str(df_pass_bp.loc[0,'Flight No.'])):
        if debug:
            print("DEBUG: Flight Number does not match.")
            print(str(df_pass_m.loc[0,'FlightNo']))
            print(str(df_pass_bp.loc[0,'Flight No.']))
        bp_valid = False
    if(str(df_pass_m.loc[0,'Seat'])!=str(df_pass_bp.loc[0,'Seat'])):
        bp_valid = False
    # Only compare first letter for Class ('E' on ticket vs 'Economy' in manifest table)
    if(str(df_pass_m.loc[0,'Class'])[0]!=str(df_pass_bp.loc[0,'Class'])[0]):
        bp_valid = False
    if(str(df_pass_m.loc[0,'From'])!=str(df_pass_bp.loc[0,'From'])):
        bp_valid = False
    if(str(df_pass_m.loc[0,'To'])!=str(df_pass_bp.loc[0,'To'])):
        bp_valid = False
    date_m = datetime.strptime(df_pass_m.loc[0, 'Date'], '%Y-%m-%d').date()
    date_bp = datetime.strptime(df_pass_bp.loc[0, 'Date'], '%B %d, %Y').date()
    if(date_m!=date_bp):
        bp_valid = False
    if(str(df_pass_m.loc[0,'BoardingTime'])!=str(df_pass_bp.loc[0,'Boarding Time'])):
        bp_valid = False
    if(bp_valid):
        if debug:
            print("Boarding Pass Validation successful for passenger " + first_name + " " + last_name)
        return True
    else:
        return False

In [306]:
def get_person_group(first_name, last_name):
    account_url = "https://computervisionproject.blob.core.windows.net"
    blob_service_client = BlobServiceClient(account_url, credential=credential)
    container = 'kiosk'
    if(first_name=="johannes" and last_name=="czylwik"):
        blob_path = 'face_video/person_group_id.txt'
        blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    elif(first_name=="avkash" and last_name=="chauhan"):
        blob_path = 'face_video/person_group_id_avkash.txt'
        blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
    downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
    person_group_id = downloader.readall()
    return person_group_id        

In [307]:
def get_face_id_from_dl(first_name, last_name):
    if(first_name=="johannes" and last_name=="czylwik"):
        dl_image = open('../material_preparation_step/IDs/ca-dl-johannes_czylwik.png', 'rb')
    elif(first_name=="avkash" and last_name=="chauhan"):
        dl_image = open('../material_preparation_step/IDs/ca-dl-avkash.png', 'rb')
    dl_faces = face_client.face.detect_with_stream(dl_image, detection_model='detection_03', recognition_model='recognition_03')
    for face in dl_faces:
        # This assumes there is only one face per image
        face_id = face.face_id
    return face_id    

In [349]:
def validate_person(manifest, first_name, last_name, debug=False):
    if ((first_name=="johannes" and last_name=="czylwik") or (first_name=="avkash" and last_name=="chauhan")):
        # Get person group that was created from thumbnails of the 30sec video
        person_group_id = get_person_group(first_name, last_name)
        if debug:
            print("DEBUG person_group_id: ",person_group_id)
        # Get face ID from the driver's license picture
        dl_face_id = get_face_id_from_dl(first_name, last_name)
        if debug:
            print("DEBUG dl_face_id: ",dl_face_id)
        identify_results = face_client.face.identify([dl_face_id], person_group_id)
        for result in identify_results:
            for candidate in result.candidates:
                match_confidence = candidate.confidence
        if debug:
            print("DEBUG match_confidence: ",match_confidence)
        if match_confidence > 0.65:
            if debug:
                print("Person Identity Validation successful for passenger " + first_name + " " + last_name)
            return True
        else:
            return False
    else:
        # No video available for other passengers - face validation failed
        return False        

In [348]:
def validate_luggage(first_name, last_name, debug=False):
    # Select a random luggage picture for lighter detection
    df_ld = get_lighter_detection_infos().sample(1)
    # Check if there is any prediction higher than a selected threshold.
    # Note: This is a "dummy prediction" anyway, since the model will pretty much detect any object as a lighter, because of the way it was trained.
    threshold=0.9
    values = df_ld.values[0]
    values = values[values!=None]
    if debug:
        print("Lighter detection confidence values:")
        print(values)
    luggage_ok = not any((pred['Lighter'] >= threshold) for pred in values)
    if luggage_ok and debug:
        print("Luggage Validation successful for passenger " + first_name + " " + last_name)
    return luggage_ok

In [340]:
def update_manifest(df_old,fn, ln, name_valid, dob_valid, person_valid, bp_valid, luggage_valid):
    df_new = df_old.copy()
    df_new.loc[(df_new['FirstName'].str.lower()==fn) & (df_manifest['LastName'].str.lower()==ln), "NameValidation"] = name_valid
    df_new.loc[(df_new['FirstName'].str.lower()==fn) & (df_manifest['LastName'].str.lower()==ln), "DoBValidation"] = dob_valid
    df_new.loc[(df_new['FirstName'].str.lower()==fn) & (df_manifest['LastName'].str.lower()==ln), "PersonValidation"] = person_valid
    df_new.loc[(df_new['FirstName'].str.lower()==fn) & (df_manifest['LastName'].str.lower()==ln), "BoardingPassValidation"] = bp_valid
    df_new.loc[(df_new['FirstName'].str.lower()==fn) & (df_manifest['LastName'].str.lower()==ln), "LuggageValidation"] = luggage_valid
    return df_new

In [368]:
def validate_passenger(manifest, first_name, last_name, debug=False):
    if debug:
        print("Starting validation for passenger " + first_name + " " + last_name + "...")
    fn_lower = first_name.lower()
    ln_lower = last_name.lower()
    name_valid = validate_name(manifest, fn_lower, ln_lower, debug)
    if(name_valid):
        dob_valid = validate_dob(manifest, fn_lower, ln_lower, debug)
        bp_valid = validate_boarding_pass(manifest, fn_lower, ln_lower, debug)
        person_valid = validate_person(manifest, fn_lower, ln_lower, debug)
        luggage_valid = validate_luggage(fn_lower, ln_lower, debug)
        new_manifest = update_manifest(manifest, fn_lower, ln_lower, name_valid, dob_valid, person_valid, bp_valid, luggage_valid)
        # Create final kiosk message
        df_pass_m = df_manifest[(df_manifest['FirstName'].str.lower()==fn_lower) & (df_manifest['LastName'].str.lower()==ln_lower)].reset_index()
        flight_no = str(df_pass_m.loc[0,'FlightNo'])
        boarding_time = str(df_pass_m.loc[0,'BoardingTime'])
        origin = str(df_pass_m.loc[0,'From'])
        destination = str(df_pass_m.loc[0,'To'])
        seat = str(df_pass_m.loc[0,'Seat'])
        if (dob_valid and bp_valid and luggage_valid and not person_valid):
            print("""Dear Mr. {0} {1},
You are welcome to flight # {2} leaving at {3} from {4} to {5}.
Your seat number is {6}, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage. 
Thanks for following the procedure.
Your identity could not be verified. Please see a customer service representative.
""".format(first_name, last_name, flight_no, boarding_time, origin, destination, seat))
        elif (bp_valid and person_valid and not luggage_valid):
            print("""Dear Mr. {0} {1},
You are welcome to flight # {2} leaving at {3} from {4} to {5}.
Your seat number is {6}, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal. 
Your identity is verified. However, your baggage verification failed, so please see a customer service representative.
""".format(first_name, last_name, flight_no, boarding_time, origin, destination, seat))
        elif (bp_valid and not person_valid and not luggage_valid):
            print("""Dear Mr. {0} {1},
You are welcome to flight # {2} leaving at {3} from {4} to {5}.
Your seat number is {6}, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal. 
Your identity could not be verified. Please see a customer service representative.
""".format(first_name, last_name, flight_no, boarding_time, origin, destination, seat))
        else:
            print("""Dear Mr. {0} {1},
You are welcome to flight # {2} leaving at {3} from {4} to {5}.
Your seat number is {6}, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage, 
thanks for following the procedure.
Your identity is verified so please board the plane.
""".format(first_name, last_name, flight_no, boarding_time, origin, destination, seat))
    else:
        # If name validation fails, don't change the manifest at all
        new_manifest = manifest.copy()
    return new_manifest

In [357]:
df_manifest_new = validate_passenger(df_manifest, "Avkash", "Chauhan", False)
df_manifest_new

Dear Mr. Avkash Chauhan,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 20A, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage, 
thanks for following the procedure.
Your identity is verified so please board the plane.



,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,False,False,False,False,False
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,False,False,False,False,False
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,False,False,False,False,False


In [380]:
df_manifest_new = validate_passenger(df_manifest_new, "Johannes", "Czylwik", True)
df_manifest_new

Starting validation for passenger Johannes Czylwik...
3-Way Person Name Validation successful for passenger johannes czylwik
Date of Birth Validation successful for passenger johannes czylwik
Boarding Pass Validation successful for passenger johannes czylwik
DEBUG person_group_id:  e1a3b214-a0df-4489-9f41-a9a88502fe16
DEBUG dl_face_id:  fac60240-a711-47f8-9091-0916d878a0d1
DEBUG match_confidence:  0.8169
Person Identity Validation successful for passenger johannes czylwik
Lighter detection confidence values:
[{'Lighter': 0.9207476} {'Lighter': 0.44547194} {'Lighter': 0.33415037}
 {'Lighter': 0.2627891} {'Lighter': 0.021372527} {'Lighter': 0.017497271}
 {'Lighter': 0.016029954} {'Lighter': 0.0135472715}
 {'Lighter': 0.011418909}]
Dear Mr. Johannes Czylwik,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 16F, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal. 
Your identi

,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,True,True,False,True,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,True,True,False,True,True
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,False


In [361]:
df_manifest_new = validate_passenger(df_manifest_new, "James", "Webb", False)
df_manifest_new

Dear Mr. James Webb,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 1A, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage. 
Thanks for following the procedure.
Your identity could not be verified. Please see a customer service representative.



,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,False,False,False,False,False
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


In [362]:
df_manifest_new = validate_passenger(df_manifest_new, "Sameer", "Kumar", False)
df_manifest_new

Dear Mr. Sameer Kumar,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 34A, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage. 
Thanks for following the procedure.
Your identity could not be verified. Please see a customer service representative.



,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


In [365]:
# Name validation fails because model reads "SKumar" on driver's license
df_manifest_new = validate_passenger(df_manifest_new, "Radha", "S. Kumar", False)
df_manifest_new

Dear Sir/Madam,
Some of the information on your ID card does not match the flight manifest data, so you cannot board the plane.
Please see a customer service representative.


,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,False,False,False,False,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


In [371]:
df_manifest_new = validate_passenger(df_manifest_new, "Libby", "Herold", False)
df_manifest_new

Dear Mr. Libby Herold,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 3D, and it is confirmed.
We have found a prohibited item in your carry-on baggage, and it is flagged for removal. 
Your identity could not be verified. Please see a customer service representative.



,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,True,True,False,True,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,False,False,False,False,False
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


In [372]:
df_manifest_new = validate_passenger(df_manifest_new, "James", "Jackson", False)
df_manifest_new

Dear Mr. James Jackson,
You are welcome to flight # 234 leaving at 10:00 AM PST from San Francisco to Chicago.
Your seat number is 25B, and it is confirmed.
We did not find a prohibited item (lighter) in your carry-on baggage. 
Thanks for following the procedure.
Your identity could not be verified. Please see a customer service representative.



,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,True,True,False,True,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,True,True,False,True,True
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


# Store new manifest in Blob Storage

In [373]:
account_url = "https://computervisionproject.blob.core.windows.net"
default_credential = DefaultAzureCredential()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient(account_url, credential=default_credential)

In [374]:
parquet_file = BytesIO()
df_manifest_new.to_parquet(parquet_file, engine = 'pyarrow')
parquet_file.seek(0)

0

In [375]:
container = 'kiosk'
blob_path = 'manifest_'+datetime.today().strftime('%Y%m%d')+'.parquet'
blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)

In [376]:
blob_client.upload_blob(data = parquet_file)

{'etag': '"0x8DC7E6BD9C6AA4B"',
 'last_modified': datetime.datetime(2024, 5, 27, 16, 41, 26, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\x11\xc2\xf91\xcd\x9a\x12\x10K\x93\x04i\xb7\x85\xc9n'),
 'client_request_id': 'f45f9a83-1c47-11ef-83b8-701ab80a14cc',
 'request_id': 'e3e2095f-701e-0006-5a54-b05674000000',
 'version': '2024-05-04',
 'version_id': None,
 'date': datetime.datetime(2024, 5, 27, 16, 41, 25, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

# Download new manifest and export to local CSV

In [377]:
blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
downloader = blob_client.download_blob(max_concurrency=1, encoding='UTF-8')
stream = BytesIO()
downloader.readinto(stream)
df_manifest_validation= pd.read_parquet(stream, engine='pyarrow')
df_manifest_validation

,Carrier,FlightNo,Class,From,To,Date,Baggage,Seat,Gate,BoardingTime,TicketNo,FirstName,LastName,DateOfBirth,Sex,NameValidation,DoBValidation,PersonValidation,BoardingPassValidation,LuggageValidation
0,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34A,G1,10:00 AM PST,34236746,Sameer,Kumar,1990-01-25,M,True,True,False,True,True
1,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,34B,G1,10:00 AM PST,34236747,Radha,S. Kumar,1994-03-05,F,False,False,False,False,False
2,UA,234,Business,San Francisco,Chicago,2022-04-20,No,1A,G1,10:00 AM PST,34236748,James,Webb,1970-12-15,M,True,True,False,True,True
3,UA,234,Business,San Francisco,Chicago,2022-04-20,No,3D,G1,10:00 AM PST,34236749,Libby,Herold,1996-02-10,F,True,True,False,True,False
4,UA,234,Economy,San Francisco,Chicago,2022-04-20,YES,25B,G1,10:00 AM PST,34236750,James,Jackson,1956-10-12,M,True,True,False,True,True
5,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,20A,G1,10:00 AM PST,34236751,Avkash,Chauhan,1990-01-01,M,True,True,True,True,True
6,UA,234,Economy,San Francisco,Chicago,2022-04-20,No,16F,G1,10:00 AM PST,34236752,Johannes,Czylwik,1989-09-05,M,True,True,True,True,True


In [378]:
df_manifest_validation.to_csv("manifest_new.csv")